In [1]:
import torch
import re
DIR = '/export/share/yutong/xgenmm/llamacpp_wd/siglip_kosmos_phi3_4k_instruct'
ckpt = torch.load(DIR + '/xgenmm.projector')

In [2]:
print(ckpt.keys())

dict_keys(['latents', 'projection.weight', 'projection.bias', 'layers.0.0.norm_media.weight', 'layers.0.0.norm_media.bias', 'layers.0.0.norm_latents.weight', 'layers.0.0.norm_latents.bias', 'layers.0.0.to_q.weight', 'layers.0.0.to_kv.weight', 'layers.0.0.to_out.weight', 'layers.0.1.0.weight', 'layers.0.1.0.bias', 'layers.0.1.1.weight', 'layers.0.1.3.weight', 'layers.1.0.norm_media.weight', 'layers.1.0.norm_media.bias', 'layers.1.0.norm_latents.weight', 'layers.1.0.norm_latents.bias', 'layers.1.0.to_q.weight', 'layers.1.0.to_kv.weight', 'layers.1.0.to_out.weight', 'layers.1.1.0.weight', 'layers.1.1.0.bias', 'layers.1.1.1.weight', 'layers.1.1.3.weight', 'layers.2.0.norm_media.weight', 'layers.2.0.norm_media.bias', 'layers.2.0.norm_latents.weight', 'layers.2.0.norm_latents.bias', 'layers.2.0.to_q.weight', 'layers.2.0.to_kv.weight', 'layers.2.0.to_out.weight', 'layers.2.1.0.weight', 'layers.2.1.0.bias', 'layers.2.1.1.weight', 'layers.2.1.3.weight', 'layers.3.0.norm_media.weight', 'layers.3

In [3]:
ckpt['latents'].shape

torch.Size([128, 1152])

In [4]:
ckpt['latents'].shape

torch.Size([128, 1152])

In [5]:
ckpt['layers.0.0.to_q.weight'].shape, ckpt['layers.0.0.to_kv.weight'].shape

(torch.Size([1536, 1152]), torch.Size([3072, 1152]))

In [6]:
ckpt['layers.0.0.to_kv.weight'].chunk(2, dim=0)[0].shape, ckpt['layers.0.0.to_kv.weight'].chunk(2, dim=0)[1].shape

(torch.Size([1536, 1152]), torch.Size([1536, 1152]))

In [7]:
for k in ckpt.keys():
    if 'layers.0' in k:
        print(k)
        
def _replace_attn_layer(key, value):
    # Check for the special case first
    if re.match(r'layers\.(\d+)\.0\.to_kv\.weight', key):
        idx = re.search(r'layers\.(\d+)\.0\.to_kv\.weight', key).group(1)
        KVweight = value.chunk(2, dim=0)
        return {f'blk.{idx}.attn.to_k.weight': KVweight[0],
                f'blk.{idx}.attn.to_v.weight': KVweight[1]
                }
    
    # Apply general replacements for other patterns
    # Define the replacement patterns
    patterns = [
        (r'layers\.(\d+)\.0\.norm_media\.(weight|bias)', r'blk.\1.attn.norm_media.\2'),
        (r'layers\.(\d+)\.0\.norm_latents\.(weight|bias)', r'blk.\1.attn.norm_latents.\2'),
        (r'layers\.(\d+)\.0\.to_q\.(weight)', r'blk.\1.attn.to_q.\2'),
        (r'layers\.(\d+)\.0\.to_out\.(weight)', r'blk.\1.attn.to_out.\2'),
        (r'layers\.(\d+)\.1\.0\.(weight|bias)', r'blk.\1.ffn.ln.\2'),
        (r'layers\.(\d+)\.1\.1\.weight', r'blk.\1.ffn.linear_up.weight'),
        (r'layers\.(\d+)\.1\.3\.weight', r'blk.\1.ffn.linear_down.weight'),
    ]
    for pattern, replacement in patterns:
        key = re.sub(pattern, replacement, key)
    
    return {key: value}

def replace_tensor_name_xgenmm_projector(ckpt):
    identifier = 'perceiver_resampler.'
    new_state_dict = {}
    for k, v in ckpt.items():
        # handel the layer
        if 'layers' in k:
            new_kvs = _replace_attn_layer(k, v)
            for new_k, new_v in new_kvs.items():
                new_state_dict[identifier+new_k] = new_v
        elif k == 'norm.weight':
            new_k = 'ln.weight'
            new_state_dict[identifier+new_k] = v
        elif k == 'norm.bias':
            new_k = 'ln.bias'
            new_state_dict[identifier+new_k] = v  
        else:
            new_state_dict[identifier+k] = v
    return new_state_dict     
        

layers.0.0.norm_media.weight
layers.0.0.norm_media.bias
layers.0.0.norm_latents.weight
layers.0.0.norm_latents.bias
layers.0.0.to_q.weight
layers.0.0.to_kv.weight
layers.0.0.to_out.weight
layers.0.1.0.weight
layers.0.1.0.bias
layers.0.1.1.weight
layers.0.1.3.weight


In [8]:
projector = replace_tensor_name_xgenmm_projector(ckpt)

In [11]:
list(projector.keys())

['perceiver_resampler.latents',
 'perceiver_resampler.projection.weight',
 'perceiver_resampler.projection.bias',
 'perceiver_resampler.blk.0.attn.norm_media.weight',
 'perceiver_resampler.blk.0.attn.norm_media.bias',
 'perceiver_resampler.blk.0.attn.norm_latents.weight',
 'perceiver_resampler.blk.0.attn.norm_latents.bias',
 'perceiver_resampler.blk.0.attn.to_q.weight',
 'perceiver_resampler.blk.0.attn.to_k.weight',
 'perceiver_resampler.blk.0.attn.to_v.weight',
 'perceiver_resampler.blk.0.attn.to_out.weight',
 'perceiver_resampler.blk.0.ffn.ln.weight',
 'perceiver_resampler.blk.0.ffn.ln.bias',
 'perceiver_resampler.blk.0.ffn.linear_up.weight',
 'perceiver_resampler.blk.0.ffn.linear_down.weight',
 'perceiver_resampler.blk.1.attn.norm_media.weight',
 'perceiver_resampler.blk.1.attn.norm_media.bias',
 'perceiver_resampler.blk.1.attn.norm_latents.weight',
 'perceiver_resampler.blk.1.attn.norm_latents.bias',
 'perceiver_resampler.blk.1.attn.to_q.weight',
 'perceiver_resampler.blk.1.attn.to

In [6]:
from transformers import AutoModel, AutoTokenizer
model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True)
resampler = model.resampler

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

configuration_minicpm.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-V-2:
- configuration_minicpm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_minicpmv.py:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

resampler.py:   0%|          | 0.00/36.0k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:

def _replace_name_resampler(s, v):
    if re.match("resampler.pos_embed", s):
        return {
            s: v,
            re.sub("pos_embed", "pos_embed_k", s): None,
        }
    if re.match("resampler.proj", s):
        return {
            re.sub("proj", "pos_embed_k", s): None, 
            re.sub("proj", "proj.weight", s): v.transpose(-1, -2).contiguous(),
        }
    if re.match("resampler.attn.in_proj_.*", s):
        return {
            re.sub("attn.in_proj_", "attn.q.", s): v.chunk(3, dim=0)[0],
            re.sub("attn.in_proj_", "attn.k.", s): v.chunk(3, dim=0)[1],
            re.sub("attn.in_proj_", "attn.v.", s): v.chunk(3, dim=0)[2],
        }
    return {s: v}

res = {}
for k in model.state_dict().keys():
    if re.match("resampler", k):
        print(k)
        temp = _replace_name_resampler(k, model.state_dict()[k])
        res.update(temp)
        print(temp.keys())
        print('===')
            

resampler.pos_embed
dict_keys(['resampler.pos_embed', 'resampler.pos_embed_k'])
===
resampler.query
dict_keys(['resampler.query'])
===
resampler.proj
dict_keys(['resampler.pos_embed_k', 'resampler.proj.weight'])
===
resampler.kv_proj.weight
dict_keys(['resampler.kv_proj.weight'])
===
resampler.attn.in_proj_weight
dict_keys(['resampler.attn.q.weight', 'resampler.attn.k.weight', 'resampler.attn.v.weight'])
===
resampler.attn.in_proj_bias
dict_keys(['resampler.attn.q.bias', 'resampler.attn.k.bias', 'resampler.attn.v.bias'])
===
resampler.attn.out_proj.weight
dict_keys(['resampler.attn.out_proj.weight'])
===
resampler.attn.out_proj.bias
dict_keys(['resampler.attn.out_proj.bias'])
===
resampler.ln_q.weight
dict_keys(['resampler.ln_q.weight'])
===
resampler.ln_q.bias
dict_keys(['resampler.ln_q.bias'])
===
resampler.ln_kv.weight
dict_keys(['resampler.ln_kv.weight'])
===
resampler.ln_kv.bias
dict_keys(['resampler.ln_kv.bias'])
===
resampler.ln_post.weight
dict_keys(['resampler.ln_post.weight']

In [9]:
res['resampler.attn.q.weight'].shape, res['resampler.attn.k.weight'].shape, res['resampler.attn.v.weight'].shape

(torch.Size([2304, 2304]), torch.Size([2304, 2304]), torch.Size([2304, 2304]))